In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cross_decomposition import PLSRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import joblib

# Chargement des données prétraitées
X_train = np.load("X_train.npy")
y_train = np.load("y_train.npy")
X_val = np.load("X_val.npy")
y_val = np.load("y_val.npy")
X_test = np.load("X_test.npy")
y_test = np.load("y_test.npy")

# Encodage One-Hot des labels pour la régression
encoder = OneHotEncoder(sparse=False)
y_train_enc = encoder.fit_transform(y_train.reshape(-1, 1))
y_val_enc = encoder.transform(y_val.reshape(-1, 1))

# Initialisation et entraînement du modèle
pls = PLSRegression(n_components=10)
pls.fit(X_train, y_train_enc)

# Prédiction sur le jeu de test
Y_pred = pls.predict(X_test)
y_pred = np.argmax(Y_pred, axis=1)

# Évaluation
print("Rapport de classification :")
print(classification_report(y_test, y_pred))

print("Matrice de confusion :")
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=encoder.categories_[0], yticklabels=encoder.categories_[0])
plt.xlabel("Prédits")
plt.ylabel("Vérité")
plt.title("Matrice de confusion - PLSRegression")
plt.show()

# Sauvegarde du modèle
joblib.dump(pls, "pls_model.joblib")
